In [ ]:
import pandas as pd
import numpy as np
from math import log2

#### 1. Fungsi Entropy

In [ ]:
def entropy(target_col):
	elements, counts = np.unique(target_col, return_counts=True)
	entropy_val = -np.sum([(counts[i] / np.sum(counts)) * log2(counts[i] / np.sum(counts)) for i in range(len(elements))])
	return entropy_val

- **Tujuan:** Menghitung entropi, yaitu ukuran ketidakpastian dalam data.
- **Input:** Kolom target (label).
- **Output:** Nilai entropi.

#### 2. Fungsi Information Gain
- **Tujuan:** Menghitung informasi gain, yaitu pengurangan ketidakpastian setelah membagi data berdasarkan atribut tertentu.
- **Input:** Data, atribut pembagi, dan kolom target.
- **Output:** Nilai informasi gain.
- **Proses:**
  - Hitung entropi awal.
  - Hitung entropi setelah pembagian berdasarkan atribut.
  - Hitung informasi gain sebagai selisih antara entropi awal dan entropi setelah pembagian.
  

In [ ]:
def info_gain(data, split_attribute_name, target_name):
	total_entropy = entropy(data[target_name])

	# Weights Entropy
	vals, counts = np.unique(data[split_attribute_name], return_counts=True)
	weighted_entropy = np.sum([
		(counts[i] / np.sum(counts)) * entropy(
			data.where(data[split_attribute_name] == vals[i]).dropna()[target_name]
		) for i in range(len(vals))
	])

	info_gain_val = total_entropy - weighted_entropy
	return info_gain_val

#### 3. Fungsi Split Information Gain
- **Tujuan:** Menghitung informasi split, yaitu ukuran distribusi data setelah pembagian berdasarkan atribut.
- **Input:** Data dan atribut pembagi.
- **Output:** Nilai informasi split (`si`).
- **Proses:**
  - Hitung entropi untuk setiap subset data berdasarkan atribut pembagi.
  - Hitung informasi split sebagai rata-rata tertimbang dari entropi subset.
  - Hitung nilai split information gain sebagai rasio antara informasi gain dan informasi split.

In [ ]:
def split_info(data, split_attribute_name):
	vals, counts = np.unique(data[split_attribute_name], return_counts=True)
	si = -np.sum([(counts[i] / np.sum(counts)) * log2(counts[i] / np.sum(counts))
		for i in range(len(vals))])
	return si

#### 4. Fungsi Gain Ratio
- **Tujuan:** Menghitung rasio informasi gain, yaitu ukuran efektivitas atribut dalam membagi data.
- **Input:** Data, atribut pembagi, dan kolom target.
- **Output:** Nilai rasio informasi gain.
- **Proses:**
  - Hitung informasi gain dan informasi split menggunakan fungsi sebelumnya.
  - Hitung rasio informasi gain sebagai rasio antara informasi gain dan informasi split.
  - Kembalikan nilai rasio informasi gain.
- **Catatan:** Jika informasi split adalah nol, kembalikan nilai nol untuk menghindari pembagian dengan nol.

In [ ]:
def gain_ratio(data, split_attribute_name, target_name):
	ig = info_gain(data, split_attribute_name, target_name)
	si = split_info(data, split_attribute_name)
	if si == 0:
		return 0
	else:
		return ig / si

#### Fungsi Rekursive dari C4.5
- **Tujuan:** Membangun pohon keputusan menggunakan algoritma C4.5 secara rekursif.
- **Input:** Data, kolom target, dan atribut yang tersisa.
- **Output:** Pohon keputusan.
- **Proses:**
  - Jika semua data memiliki label yang sama, kembalikan label tersebut sebagai daun pohon.
  - Jika tidak ada atribut tersisa, kembalikan label mayoritas dari data sebagai daun pohon.
  - Hitung rasio informasi gain untuk setiap atribut.
  - Pilih atribut dengan rasio informasi gain tertinggi sebagai atribut pembagi.
  - Buat simpul pohon untuk atribut pembagi.
  - Bagi data berdasarkan nilai atribut pembagi dan panggil fungsi rekursif untuk setiap subset data.
  - Kembalikan pohon keputusan yang telah dibangun.
- **Catatan:** Fungsi ini menggunakan struktur data `Node` untuk merepresentasikan simpul pohon keputusan.

In [ ]:
def c45(data, original_data, features, target_attribute_name, parent_node_class=None):
	# Base conditions
	if len(np.unique(data[target_attribute_name])) <= 1:
		return np.unique(data[target_attribute_name])[0]
	elif len(data) == 0:
		return np.unique(original_data[target_attribute_name])[
			np.argmax(np.unique(original_data[target_attribute_name], return_counts=True)[1])
		]
	elif len(features) == 0:
		return parent_node_class
	else:
		# Kelas mayoritas pada node induk
		parent_node_class = np.unique(data[target_attribute_name])[
			np.argmax(np.unique(data[target_attribute_name], return_counts=True)[1])
		]

		# Pilih atribut dengan Gain Ratio tertinggi
		gain_ratios = [gain_ratio(data, feature, target_attribute_name) for feature in features]
		best_feature_index = np.argmax(gain_ratios)
		best_feature = features[best_feature_index]

		tree = {best_feature: {}}

		# Nilai feature dan subtree
		for value in np.unique(data[best_feature]):
			sub_data = data.where(data[best_feature] == value).dropna()
			subtree = c45(
				sub_data,
				original_data,
				[feat for feat in features if feat != best_feature],
				target_attribute_name, 
				parent_node_class
			)
			tree[best_feature][value] = subtree

		return tree

# Fungsi untuk memprediksi kelas
def predict(query, tree):
	for attribute in query:
		if attribute in tree:
			try:
				result = tree[attribute][query[attribute]]
			except KeyError:
				return "Unknown"
			
			if isinstance(result, dict):
				return predict(query, result)
			else:
				return result

### Contoh Implementasi

In [ ]:
# Contoh data
data = {
	'Outlook': ['Sunny', 'Sunny', 'Overcast', 'Rain', 'Rain', 'Rain', 'Overcast', 'Sunny', 'Sunny', 'Rain', 'Sunny', 'Overcast', 'Overcast', 'Rain'],
	'Temperature': ['Hot', 'Hot', 'Hot', 'Mild', 'Cool', 'Cool', 'Cool', 'Mild', 'Cool', 'Mild', 'Mild', 'Mild', 'Hot', 'Mild'],
	'Humidity': ['High', 'High', 'High', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'Normal', 'Normal', 'High', 'Normal', 'High'],
	'Wind': ['Weak', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Weak', 'Weak', 'Strong', 'Strong', 'Weak', 'Strong'],
	'PlayTennis': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No']
}

df = pd.DataFrame(data)
df

In [ ]:
# Panggil fungsi C4.5
tree = c45(
	df,
	df,
	df.columns[:-1].tolist(),
	'PlayTennis'
)
print("Decision Tree:")
print(tree)

In [ ]:
# Prediksi dengan contoh data
query = {'Outlook': 'Sunny', 'Temperature': 'Cool', 'Humidity': 'High', 'Wind': 'Strong'}
predicted_class = predict(query, tree)
print("\nPredicted class for query:")
print(query)
print("Predicted class: {}".format(predicted_class))